# code to retrack data from old btrack versions

First, make sure you have the latest version of btrack installed:

```sh
pip install --upgrade -e git+git://github.com/quantumjot/BayesianTracker.git
```

In [1]:
import os
import napari
import btrack
import numpy as np

In [2]:
import glob

In [3]:
glob.glob('/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0806/Pos6/Pos6_aligned/HDF/*')

['/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0806/Pos6/Pos6_aligned/HDF/retracked_new.h5',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0806/Pos6/Pos6_aligned/HDF/segmented_old.hdf5',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0806/Pos6/Pos6_aligned/HDF/segmented.h5']

In [8]:
PATH = "/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0806/Pos6/Pos6_aligned/HDF"
OLDFILE = os.path.join(PATH, 'segmented_old.hdf5')
NEWFILE = os.path.join(PATH, 'retracked_new_2.h5')

In [5]:
obj_types = ['obj_type_1', 'obj_type_2']
obj_models = [
    'MDCK_config_new.json',
    'MDCK_config_new.json'
]

### grab the segmentation

In [6]:
with btrack.dataio.HDF5FileHandler(OLDFILE, 'r') as hdf:
    segmentation = hdf.segmentation

[INFO][2021/07/30 07:12:35 PM] Opening HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0806/Pos6/Pos6_aligned/HDF/segmented_old.hdf5...
[INFO][2021/07/30 07:12:35 PM] Loading segmentation
[INFO][2021/07/30 07:12:42 PM] Closing HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0806/Pos6/Pos6_aligned/HDF/segmented_old.hdf5


### iterate over the cell types, reorganise the data and retrack, writing out the files

In [ ]:
for obj_type, obj_model in zip(obj_types, obj_models):

    with btrack.dataio.HDF5FileHandler(OLDFILE, 'r', obj_type=obj_type) as hdf:
        objects = hdf.objects
        area = hdf._hdf['objects'][obj_type]['area'][:]
        
    for idx, obj in enumerate(objects):
        obj.properties = {'area': area[idx]}
        
    # make a filtered list of objects for tracking
    filtered_objects = [o for o in objects if o.properties['area']>100.]
    
    with btrack.BayesianTracker() as tracker:

        # configure the tracker using a config file
        tracker.configure_from_file(
            os.path.join("/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/Bayesian_config/", obj_model)
        )
        tracker.max_search_radius = 50

        # append the objects to be tracked
        tracker.append(filtered_objects)

        # set the volume
        tracker.volume=((0, 1600), (0, 1200), (-1e5, 1e5))

        # track them (in interactive mode)
        tracker.track_interactive(step_size=100)

        # generate hypotheses and run the global optimizer
        tracker.optimize()

        tracker.export(
            NEWFILE, 
            obj_type=obj_type, 
            filter_by='area>100'
        )
        
        tracks = tracker.tracks

[INFO][2021/07/30 07:18:17 PM] Opening HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0806/Pos6/Pos6_aligned/HDF/segmented_old.hdf5...
[INFO][2021/07/30 07:18:17 PM] Loading objects/obj_type_1 (95943, 5) (95943 filtered: None)
[INFO][2021/07/30 07:18:17 PM] Closing HDF file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/GV0806/Pos6/Pos6_aligned/HDF/segmented_old.hdf5
[INFO][2021/07/30 07:18:18 PM] Loaded btrack: /home/nathan/anaconda3/envs/napari/lib/python3.7/site-packages/btrack/libs/libtracker.so
[INFO][2021/07/30 07:18:18 PM] btrack (v0.4.0) library imported
[INFO][2021/07/30 07:18:18 PM] Setting max XYZ search radius to: 100
[INFO][2021/07/30 07:18:18 PM] Starting BayesianTracker session
[INFO][2021/07/30 07:18:18 PM] Loading configuration file: /run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Giulia/Bayesian_config/MDCK_config_new.json
[INFO][202

[INFO][2021/07/30 07:19:05 PM] Loading motion model: b'MDCK_motion_Kristina'
[INFO][2021/07/30 07:19:05 PM] Setting max XYZ search radius to: 50
[INFO][2021/07/30 07:19:05 PM] Objects are of type: <class 'list'>
[INFO][2021/07/30 07:19:05 PM] Set volume to ((0, 1600), (0, 1200), (-100000.0, 100000.0))
[INFO][2021/07/30 07:19:05 PM] Starting tracking... 
[INFO][2021/07/30 07:19:06 PM] Tracking objects in frames 0 to 99 (of 1191)...
[INFO][2021/07/30 07:19:06 PM]  - Timing (Bayesian updates: 1.66ms, Linking: 0.34ms)
[INFO][2021/07/30 07:19:06 PM]  - Probabilities (Link: 1.00000, Lost: 1.00000)
[INFO][2021/07/30 07:19:06 PM]  - Stats (Active: 79, Lost: 444, Conflicts resolved: 377)
[INFO][2021/07/30 07:19:06 PM] Tracking objects in frames 100 to 199 (of 1191)...
[INFO][2021/07/30 07:19:06 PM]  - Timing (Bayesian updates: 2.12ms, Linking: 0.30ms)
[INFO][2021/07/30 07:19:06 PM]  - Probabilities (Link: 1.00000, Lost: 0.99229)
[INFO][2021/07/30 07:19:06 PM]  - Stats (Active: 105, Lost: 808, C

### finally, write the segmentation back to the file

In [ ]:
with btrack.dataio.HDF5FileHandler(NEWFILE, 'a') as hdf:
    hdf.write_segmentation(segmentation)
    print('btrack version: ', hdf._hdf['tracks']['obj_type_1'].attrs['version'])

In [ ]:
objects[0]